In [14]:
import pandas as pd
import math
import logging
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [9]:
data = pd.read_csv("parser_output/extracted_features_split_3_7.csv")
data = data.dropna()

In [10]:
y = data[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
x = data[data.columns[1:len(data.columns)-12]]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.7)

In [11]:
#lINEAR REGRESSION - 
reg_model = LinearRegression().fit(X_train, y_train)
ypred_train = reg_model.predict(X_train)
ypred_test = reg_model.predict(X_test)
print("LINEAR REGRESSION")
print("Train Score: ")
print(r2_score(y_train, ypred_train))
print("Test Score: ")
print(r2_score(y_test, ypred_test))

LINEAR REGRESSION
Train Score: 
1.0
Test Score: 
-29.30024986822348


In [12]:
print(ypred_test, '\n\n\n')
print(ypred_test[0])

[[ 8.80519912e-01 -1.30215355e-01 -2.00908315e+01 ... -3.41310741e-01
   2.42025064e-01  1.86613938e+02]
 [ 1.24335129e+00  1.99433220e-01 -1.79766603e+01 ...  2.14288359e-02
  -3.98414154e-01  2.17127951e+02]
 [ 4.65918534e-01 -1.09349458e-01 -1.89082686e+01 ...  2.93421374e-01
  -1.87447874e+00  2.54340469e+02]
 ...
 [ 8.96992044e-01  6.57714617e-01  8.11629843e-01 ...  5.58951827e-01
   8.06016537e-01  1.80903202e+02]
 [ 8.37952043e-01 -1.09711639e-01 -2.34524785e+01 ...  1.68459591e-01
  -4.13046967e-01  1.08741206e+02]
 [ 1.01526882e+00  4.15053345e-02 -4.19289041e+00 ...  7.58863631e-02
   1.48213148e-02  2.05433556e+02]] 



[ 8.80519912e-01 -1.30215355e-01 -2.00908315e+01  7.56437905e-02
  7.62148299e-01  8.05315030e-02 -3.41310741e-01  2.42025064e-01
  1.86613938e+02]


In [15]:
# Instantiate Spotify Client
logger = logging.getLogger()
logging.basicConfig()
CLIENT_ID="9793440f0a5047c59c70bcfcf91ad589"
CLIENT_SECRET= "b66dc3a5f9f34207bebee32a25745368"
REDIRECT_URL="http://localhost/"
client_credentials_manager = SpotifyClientCredentials(client_id = CLIENT_ID, client_secret = CLIENT_SECRET)
oAuth = SpotifyOAuth(client_id = CLIENT_ID, client_secret = CLIENT_SECRET, redirect_uri = REDIRECT_URL, scope = 'user-modify-playback-state')
sp = spotipy.Spotify(auth_manager =oAuth)

In [58]:
def get_song(name_song, name_artist):
    results = sp.search(q=name_song + ' '+name_artist, type='track')
    items = results['tracks']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None

# Spotify Song Features
def get_audio_features(song_name, artist_name):

    song = get_song(song_name, artist_name)

    #if get request didnt get anything
    if(song is None):
        return None

    song_features = sp.audio_features([song['uri']])

    audio_feature_list = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                          'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

    audio_feat = []
    
    audio_feat.append(song_features[0].get('danceability'))
    audio_feat.append(song_features[0].get('energy'))
    audio_feat.append(song_features[0].get('loudness'))
    audio_feat.append(song_features[0].get('speechiness'))
    audio_feat.append(song_features[0].get('acousticness'))
    audio_feat.append(song_features[0].get('instrumentalness'))
    audio_feat.append(song_features[0].get('liveness'))
    audio_feat.append(song_features[0].get('valence'))
    audio_feat.append(song_features[0].get('tempo'))

    return audio_feat

In [59]:
song_artist_pairs = {
    '1':('thats_what_i_like','bruno_mars'),
    '2':('humble','kendrick_lamar'),
    '3':('skeletons','keshi'),
    '4':('slow_dancing_in_the_dark','joji'),
    '5':('lite_spots','kaytranada'),
    '6':('woman','doja_cat'),
    '7':('get_up','ciara'),
    '8':('throwin_elbows','excision'),
    '9':('power','little mix'),
    '10':('peaches','justin_bieber'),
    '11':('knife_talk','drake'),
    '12':('fool_around','yas'),
    '13':('levitating','dua_lipa'),
    '14':('feed_the_fire','lucky_daye'),
    '15':('easily','bruno_major'),
    '16':('good_4_u','olivia_rodrigo'),
    '17':('all_i_wanna_do','jay_park'),
    '18':('sad_girlz_luv_money','amaarae'),
    '19':('tik_tok','kesha'),
    '20':('ymca','village_people'),
    '21':('intuition_interlude','jamie_foxx'),
    '22':('kilby_girl','the_backseat_lovers'),
    '23':('a_thousand_miles','vanessa_carlton')
}                

In [74]:
song_artist_features = []

# Extract all features
for key in song_artist_pairs.keys():

    song_name = song_artist_pairs[key][0]
    artist_name = song_artist_pairs[key][1]

    try:
        audio_feat = get_audio_features(song_name, artist_name)
    except:
        oAuth = SpotifyOAuth(client_id = CLIENT_ID, client_secret = CLIENT_SECRET, redirect_uri = REDIRECT_URL, scope = 'user-modify-playback-state')
        sp = spotipy.Spotify(auth_manager =oAuth)
        audio_feat = get_audio_features(song_name, artist_name)
    if audio_feat:
        song_artist_features.append((song_name, artist_name, audio_feat))

In [75]:
for i in range(len(song_artist_features)):
    print(song_artist_features[i])

('thats_what_i_like', 'bruno_mars', [0.853, 0.56, -4.961, 0.0406, 0.013, 0, 0.0944, 0.86, 134.066])
('humble', 'kendrick_lamar', [0.908, 0.621, -6.638, 0.102, 0.000282, 5.39e-05, 0.0958, 0.421, 150.011])
('skeletons', 'keshi', [0.719, 0.265, -12.782, 0.0754, 0.296, 0.0865, 0.108, 0.273, 80.021])
('slow_dancing_in_the_dark', 'joji', [0.515, 0.479, -7.458, 0.0261, 0.544, 0.00598, 0.191, 0.284, 88.964])
('lite_spots', 'kaytranada', [0.884, 0.549, -11.683, 0.471, 0.0346, 0.0698, 0.112, 0.394, 120.461])
('woman', 'doja_cat', [0.824, 0.764, -4.175, 0.0854, 0.0888, 0.00294, 0.117, 0.881, 107.998])
('get_up', 'ciara', [0.964, 0.595, -6.887, 0.109, 0.0248, 5.14e-06, 0.0405, 0.629, 128.593])
('throwin_elbows', 'excision', [0.574, 0.94, -3.038, 0.569, 0.00317, 0.0431, 0.113, 0.362, 149.988])
('power', 'little mix', [0.566, 0.855, -4.016, 0.211, 0.0372, 0.000157, 0.181, 0.517, 172.966])
('peaches', 'justin_bieber', [0.677, 0.696, -6.181, 0.119, 0.321, 0, 0.42, 0.464, 90.03])
('knife_talk', 'drake'

In [78]:
def find_diffs(feature1, feature2):
    diffs = 0
    for i in range(len(feature1)):
        diffs += abs(feature1[i] - feature2[i])
    return diffs

def find_diffs_sq(feature1, feature2):
    diffs = 0
    for i in range(len(feature1)):
        diffs += (feature1[i] - feature2[i])**2
    return diffs

def find_closest_song(features):
    
    minDistance = sum(abs(features))
    minIndex = 0
        
    for i in range(len(song_artist_features)):
        diffs = find_diffs(features, song_artist_features[i][2])
        if diffs < minDistance:
            minDistance = diffs
            minIndex = i
            
    return minIndex
        

In [80]:
maxDiffs = find_closest_song(ypred_test[0])
print(song_artist_features[i][0], song_artist_features[i][1])

a_thousand_miles vanessa_carlton
